## Validate with pydantic

In [16]:
from pydantic import BaseModel, ValidationError

# Student is a normal class but is also a pydantic BaseModel as we inherit from it
class Student(BaseModel):
    name: str
    gender: str
    age: int

s1 = Student(name="Beda", gender="F", age=8)
s1

Student(name='Beda', gender='F', age=8)

In [14]:
s1.age = 20
s1.name = "Doddo"
s1

Student(name='Doddo', gender='F', age=20)

In [15]:
try:
    Student(name=2.714, gender=8, age=-8)
except ValidationError as err:
    print(err)

2 validation errors for Student
name
  Input should be a valid string [type=string_type, input_value=2.714, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=8, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [21]:
from pydantic import Field
from typing import Literal

class Student(BaseModel):
    name: str
    gender: str = Literal["M", "F"]
    age: int = Field(gt=-1, lt=125)  # age must be between 0 and 124

try:
    Student(name = "Bibbi", gender = "F", age = -1)
except ValidationError as err:
    print(err)

1 validation error for Student
age
  Input should be greater than -1 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


In [25]:
s2 = Student(name = "Bibbi", gender = "F", age = 1)

In [26]:
Student(name = "Bibbi", gender = "F", age = "1")

Student(name='Bibbi', gender='F', age=1)

## Serialization and deserialization

- serialization: convert object -> json
- deserialization: json -> object

### model_dump_json(): 
- returns a JSON string (str). 
- It serializes the model to JSON (converts datetimes, enums, etc. to JSON-friendly representations).

In [27]:
s2.model_dump_json()

'{"name":"Bibbi","gender":"F","age":1}'

### model_dump(): 
- returns a Python object (usually a dict) with the model data. 
- Default mode is "python" so values remain Python-native (e.g. datetimes stay datetime objects). 
- Useful when you want to inspect or further manipulate the data.

In [28]:
s2.model_dump()

{'name': 'Bibbi', 'gender': 'F', 'age': 1}

In [40]:
import json

with open("housing.json", "r", encoding='utf-8') as file:
    json_data = json.load(file)

json_data

[{'address': 'Sveavägen 98, Norrmalm',
  'price': 5200000,
  'area': 34,
  'rooms': 1.5},
 {'address': 'Hornsgatan 72, Södermalm',
  'price': 7600000,
  'area': 52,
  'rooms': 2},
 {'address': 'Värtavägen 12, Östermalm',
  'price': 11200000,
  'area': 78,
  'rooms': 3},
 {'address': 'Götgatan 112, Södermalm',
  'price': 8900000,
  'area': 65,
  'rooms': 2.5},
 {'address': 'Hälsingegatan 4, Vasastan',
  'price': 6700000,
  'area': 44,
  'rooms': 1.5}]

In [39]:
class House(BaseModel):
    address: str
    price: float    
    area: float
    rooms: float

In [41]:
json_data[0]

{'address': 'Sveavägen 98, Norrmalm',
 'price': 5200000,
 'area': 34,
 'rooms': 1.5}

In [44]:
# use kwargs to unpack the dictionary into the model
House(**json_data[0])

House(address='Sveavägen 98, Norrmalm', price=5200000.0, area=34.0, rooms=1.5)

In [47]:
[House(**item) for item in json_data]

[House(address='Sveavägen 98, Norrmalm', price=5200000.0, area=34.0, rooms=1.5),
 House(address='Hornsgatan 72, Södermalm', price=7600000.0, area=52.0, rooms=2.0),
 House(address='Värtavägen 12, Östermalm', price=11200000.0, area=78.0, rooms=3.0),
 House(address='Götgatan 112, Södermalm', price=8900000.0, area=65.0, rooms=2.5),
 House(address='Hälsingegatan 4, Vasastan', price=6700000.0, area=44.0, rooms=1.5)]

In [48]:
from pydantic import BaseModel, Field
class House(BaseModel):
    address: str
    price: int = Field(lt = 10_000_000)
    area: float
    rooms: float

In [49]:
House(address="....", price=8391, area=32, rooms=2.0)

House(address='....', price=8391, area=32.0, rooms=2.0)

In [50]:
# House(address="....", price=5200..., area=34.0, rooms=1.5)
House(**json_data[0])

House(address='Sveavägen 98, Norrmalm', price=5200000, area=34.0, rooms=1.5)

In [51]:
[House(**house_data) for house_data in json_data]

ValidationError: 1 validation error for House
price
  Input should be less than 10000000 [type=less_than, input_value=11200000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than